# Gradient Descent for Multilayer-Perceptrons

## Importing the needed libraries

In [97]:
import numpy as np
import pandas as pd

## Prepare our dataset

In [98]:
df = pd.read_csv('binary.csv')
df

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4
...,...,...,...,...
395,0,620,4.00,2
396,0,560,3.04,3
397,0,460,2.63,2
398,0,700,3.65,2


In [99]:
df['rank'].nunique()

4

## Perform One-Hot Encoding

In [100]:
data = pd.get_dummies(df,columns = ['rank'])
data

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,380,3.61,0,0,1,0
1,1,660,3.67,0,0,1,0
2,1,800,4.00,1,0,0,0
3,1,640,3.19,0,0,0,1
4,0,520,2.93,0,0,0,1
...,...,...,...,...,...,...,...
395,0,620,4.00,0,1,0,0
396,0,560,3.04,0,0,1,0
397,0,460,2.63,0,1,0,0
398,0,700,3.65,0,1,0,0


## Standarize features

In [101]:
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    df.loc[:, field] = (df[field] - mean) / std

## Split off random 20% of the data for testing

In [102]:
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data) * 0.8), replace=False)
data, test_data = data.loc[sample], data.drop(sample)

## Split into features and targets

In [103]:
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

## Begin our Model

### Defining the Sigmoid function for activations

In [104]:
# Calculate the sigmoid function
def sigmoid(x):
    return 1/(1 + np.exp(-x))

# Derivative of the sigmoid function
def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))

## Hyperparameters

In [105]:
np.random.seed(21)
n_hidden = 2  # number of hidden units
epochs = 900
learnrate = 0.005
n_records, n_features = features.shape
last_loss = None

## Initialize weights

In [106]:
weights_input_hidden =  np.random.normal( scale=1 / n_features ** .5 , size=(n_features, n_hidden) )
weights_hidden_output = np.random.normal( scale=1 / n_features ** .5 , size=n_hidden )

## Executing Gradient steps

In [107]:
for e in range(epochs):
    del_w_input_hidden = np.zeros(weights_input_hidden.shape)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)
    for x, y in zip(features.values, targets):
        ## Forward pass ##
        # Calculating the output
        hidden_input = np.dot(x, weights_input_hidden)
        hidden_output = sigmoid(hidden_input)
        output = sigmoid(np.dot(hidden_output, weights_hidden_output))

        ## Backward pass ##
        # Calculating the network's prediction error
        error = y - output
        # TODO: Calculating error term for the output unit
        output_error_term = error  *output*  (1 - output)

        ## Propagate errors to hidden layer
        # Calculating the hidden layer's contribution to the error
        hidden_error = np.dot(output_error_term, weights_hidden_output)
        # Calculating the error term for the hidden layer
        hidden_error_term = hidden_error  *hidden_output*  (1 - hidden_output)
        # Update the change in weights
        del_w_hidden_output += output_error_term * hidden_output
        del_w_input_hidden += hidden_error_term * x[:, None]

    # Updating weights
    weights_input_hidden += learnrate * del_w_input_hidden / n_records
    weights_hidden_output += learnrate * del_w_hidden_output / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        hidden_output = sigmoid(np.dot(x, weights_input_hidden))
        out = sigmoid(np.dot(hidden_output,
                             weights_hidden_output))
        loss = np.mean((out - targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

Train loss:  0.25000049341740294
Train loss:  0.2500001166875275
Train loss:  0.250000049757889
Train loss:  0.25000002694480195
Train loss:  0.2500000166427851
Train loss:  0.25000001117851356
Train loss:  0.2500000079589956
Train loss:  0.25000000591559185
Train loss:  0.2500000045445667
Train loss:  0.2500000035841167


## Calculate accuracy on test data

In [108]:
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
predictions = out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Prediction accuracy: 0.350
